# Align and remove blanks

Adapted from Giulia Vallardi's ImageJ macro, this notebook removes any blank frames from timelapse experiments and aligns the images. 

"Fiji macro to remove over- and under-exposed images, and align the image stacks

The settings for the alignments are: 
registration by Translation > only modify XY coordinates
Shrinkage constrain activated (this model allows a better registration based on all images, not using a reference image. It is more time consuming though)
Transform matrices are saved during registration and then applied to the other channels during transformation."

In [30]:
import os
import glob
import enum
import numpy as np
from pystackreg import StackReg
from skimage import io
from tqdm import tqdm
from daskoctopus import DaskOctopusLiteLoader
from skimage import transform as tf

In [3]:
import timeit
def start():
    global start_time
    start_time = timeit.default_timer()

def stop():
    elapsed = timeit.default_timer() - start_time
    print('elapsed time:', elapsed)

# Find images, organise into raw folder and load using dask octo

In [4]:
### define root directory and specific experiment and location (will later make iterable)
root_dir = '/home/nathan/data/kraken/commitment/test/'
expt = "MK0003"
pos = "PosX"

In [5]:
### create new subdir of for raw files and move them all there
if not os.path.exists(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw')):
    os.mkdir(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw'))
    files = sorted(glob.glob(os.path.join(root_dir, f'{expt}/{pos}/*.tif')))
    for file in files:
        os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_raw'))

In [7]:
### pre load files from raw file dir 
images = DaskOctopusLiteLoader(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw'))

Using cropping: None


In [39]:
images['irfp']

dask.array<stack, shape=(1200, 1352, 1688), dtype=float32, chunksize=(1, 1352, 1688), chunktype=numpy.ndarray>

# Find blank or overexposed images and display average channel brightness

In [9]:
start()
# pixel range criteria
max_pixel, min_pixel = 200, 2

### find mean values### pre load files from raw file dir 
images = DaskOctopusLiteLoader(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw'))
gfp_mean_array = np.mean(images['gfp'], axis = (1,2)).compute()
rfp_mean_array = np.mean(images['rfp'], axis = (1,2)).compute()
irfp_mean_array = np.mean(images['irfp'], axis = (1,2)).compute()
bf_mean_array = np.mean(images['brightfield'], axis = (1,2)).compute()

# find blanks / overexposed and create dodgy frame list
dodgy_frame_list = set([])
for i, array in enumerate([gfp_mean_array, rfp_mean_array, irfp_mean_array, bf_mean_array]):
    for frame, mean_value in enumerate(array):
        if max_pixel < mean_value or mean_value < min_pixel:
            dodgy_frame_list.add(frame)
dodgy_frame_list = list(dodgy_frame_list)

print('Average channel brightness for selection of reference image:')
print('gfp:', np.mean(gfp_mean_array), 'rfp:', np.mean(rfp_mean_array), 'irfp:', np.mean(irfp_mean_array), 'bf:', np.mean(bf_mean_array))
stop()

Average channel brightness for selection of reference image:
gfp: 62.79905 rfp: 6.4401646 irfp: 76.01225 bf: 28.949467
elapsed time: 74.95976201398298


# Select reference image to base alignment around

In [11]:
channels = images.channels
channels

[<Channels.BRIGHTFIELD: 0>,
 <Channels.GFP: 1>,
 <Channels.RFP: 2>,
 <Channels.IRFP: 3>]

In [13]:
reference_channel = images.channels[3]
reference_channel.name

'IRFP'

## Set cropped area of reference image to base alignment around (whole image struggles to compute)

In [44]:
reference_image = images[reference_channel.name]

In [17]:
crop_area = 500
reference_image = reference_image[:,int((reference_image.shape[2]-crop_area)/2):int(reference_image.shape[2]-(reference_image.shape[2]-crop_area)/2),int((reference_image.shape[1]-crop_area)/2):int(reference_image.shape[1]-(reference_image.shape[1]-crop_area)/2)].compute()

elapsed time: 2.426488220982719


In [41]:
reference_image.shape

(1200, 500, 500)

# Register alignment and save out

In [20]:
# create operator using transformation type (translation)
sr = StackReg(StackReg.TRANSLATION) 

# register each frame to the previous as transformation matrices/tensor
transform_tensor = sr.register_stack(reference_image, reference = 'previous').astype(np.uint8)

# save out transform tensor
np.save(os.path.join(root_dir, f'{expt}/{pos}/{reference_channel.name.lower()}_transform_tensor.npy'), transform_tensor)

/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


In [21]:
transform_tensor.shape

(1200, 3, 3)

# Apply transformation matrix to all channels and save out images (~11min) -- add ignore low contrast message

In [38]:
### iterating over channels
# create aligned image dir if does not exist 
if not os.path.exists(os.path.join(root_dir, f'{expt}/{pos}/{pos}_aligned')):
    os.mkdir(os.path.join(root_dir, f'{expt}/{pos}/{pos}_aligned'))
# iterate over channels
for channel in channels:
    print('Aligning', channel.name.lower(), 'channel', channel.value, '/', len(channels))
    #iterate over all images in channel
    for i in tqdm(range(len(images[channel.name]))):
        # skip dodgy frames and don't save out into aligned folder
        if i in dodgy_frame_list:
            continue
        # load specific transform matrix for that frame
        transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],rotation = None)
        # transform image
        transformed_image = (tf.warp(images[channel.name][i,...], transform_matrix)*255).astype(np.uint8)
        # set transformed image pathname by editing base dir
        fn = images.files(channel.name)[i].replace('_raw', '_aligned')
        # save trans image out
        io.imsave(fn, transformed_image)

Aligning brightfield channel 0 / 4


100%|██████████| 1200/1200 [02:34<00:00,  7.76it/s]


Aligning gfp channel 1 / 4


100%|██████████| 1200/1200 [02:56<00:00,  6.79it/s]


Aligning rfp channel 2 / 4


...

 33%|███▎      | 394/1200 [00:55<01:25,  9.46it/s]<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000394_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
 33%|███▎      | 395/1200 [00:55<01:27,  9.21it/s]<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000395_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000396_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
 33%|███▎      | 397/1200 [00:55<01:28,  9.09it/s]<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000397_z000.tif is a low cont

 41%|████      | 491/1200 [01:08<01:10, 10.02it/s]<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000491_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000492_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
 41%|████      | 493/1200 [01:08<01:10,  9.99it/s]<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000493_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
<ipython-input-38-491982e16af0>:14: UserWarning: /home/nathan/data/kraken/commitment/test/MK0003/PosX/PosX_aligned/img_channel002_position001_time000000494_z000.tif is a low contrast image
  io.imsave(fn, transformed_image)
 41%

Aligning irfp channel 3 / 4


100%|██████████| 1200/1200 [02:40<00:00,  7.48it/s]

elapsed time: 661.7332741690043


# Try out mean measure and alignment in this loop and iterate over all expts and pos?

In [ ]:
### define root directory and specific experiment and location (will later make iterable)
root_dir = '/home/nathan/data/kraken/commitment/test/'

for expt in expt_list:
    for pos in pos_list:

        ### create new subdir of for raw files and move them all there
        if not os.path.exists(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw')):
            os.mkdir(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw'))
            files = sorted(glob.glob(os.path.join(root_dir, f'{expt}/{pos}/*.tif')))
            for file in files:
                os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_raw'))

        ### pre load files from raw file dir 
        images = DaskOctopusLiteLoader(os.path.join(root_dir, f'{expt}/{pos}/{pos}_raw'))

        # pixel range criteria
        max_pixel, min_pixel = 200, 2

        ### find mean values
        gfp_mean_array = np.mean(images['gfp'], axis = (1,2)).compute()
        rfp_mean_array = np.mean(images['rfp'], axis = (1,2)).compute()
        irfp_mean_array = np.mean(images['irfp'], axis = (1,2)).compute()
        bf_mean_array = np.mean(images['brightfield'], axis = (1,2)).compute()

        # find blanks / overexposed and create dodgy frame list
        dodgy_frame_list = set([])
        for i, array in enumerate([gfp_mean_array, rfp_mean_array, irfp_mean_array, bf_mean_array]):
            for frame, mean_value in enumerate(array):
                if max_pixel < mean_value or mean_value < min_pixel:
                    dodgy_frame_list.add(frame)
        dodgy_frame_list = list(dodgy_frame_list)

        print('Average channel brightness for selection of reference image:')
        print('gfp:', np.mean(gfp_mean_array), 'rfp:', np.mean(rfp_mean_array), 'irfp:', np.mean(irfp_mean_array), 'bf:', np.mean(bf_mean_array))

        ### iterating over channels
        # create aligned image dir if does not exist 
        if not os.path.exists(os.path.join(root_dir, f'{expt}/{pos}/{pos}_aligned')):
            os.mkdir(os.path.join(root_dir, f'{expt}/{pos}/{pos}_aligned'))

        # pick reference image and create trans tensor


        # iterate over channels
        for channel in channels:
            print('Aligning', channel.name.lower(), 'channel', channel.value, '/', len(channels))
            #iterate over all images in channel
            for i in tqdm(range(len(images[channel.name]))):
                # skip dodgy frames and don't save out into aligned folder
                if i in dodgy_frame_list:
                    continue
                # load specific transform matrix for that frame
                transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],rotation = None)
                # transform image
                transformed_image = (tf.warp(images[channel.name][i,...], transform_matrix)*255).astype(np.uint8)
                # set transformed image pathname by editing base dir
                fn = images.files(channel.name)[i].replace('_raw', '_aligned')
                # save trans image out
                io.imsave(fn, transformed_image)

# Compile stacks and save out?

In [ ]:
start()
aligned_images = DaskOctopusLiteLoader(os.path.join(root_dir, f'{expt}/{pos}/{pos}_aligned'))
stop()